#**Example Machine Learning application of learning a crop type classifiction model using training data from Radiant ML Hub**

This Python notebook takes you through an application of learning a classification model for classifying crop types against field IDs in a satellite image.

The notebook is based on a starter notebook provided for the 2020 ICLR workshop challenge at Zindi on using earth observation data for classifying crop types during the growing season: https://zindi.africa/competitions/iclr-workshop-challenge-2-radiant-earth-computer-vision-for-crop-recognition

---
This is a high level overview of the notebook:

- Download data (tif files and label and field information) from Radiant ML Hub
- Do data preprocessing by sampling the satellite images to get the data into an easy tabular form 
- Learn a classification model on this data, and experiment with grouping by field or doing pixel-wise classifications.

---

## **Download Data from Radiant ML Hub**

At the end of this section you should have downloaded all images and labels of the crop type dataset that constitutes satellite imagery over western Kenya labeled with field IDs and respective crop types.

The code in this section is essentially a clone from a starter notebook provided by Radiant Earth for downloading satellite images labeled with field IDs and crop types: https://github.com/radiantearth/mlhub-tutorials/tree/master/notebooks/2020%20CV4A%20Crop%20Type%20Challenge

In this section we will:

- install the new radiant mlhub client.
- import libraries that we use to access and get information from the Radiant ML Hub API.
- specify authentication details and API properties that we shall use whenever we access the API.
- specify the download location where the dataset will be downloaded.
- define functions to use for donwloading labels and images.
- specify collection IDs and download the respective labels and images.


###**Install Radiant MLHub client**

In [ ]:
!pip install radiant_mlhub

###**Import necessary packages**

In [2]:
#import urllib.parse
#import re
#from pathlib import Path
#import itertools as it
#from functools import partial
#from concurrent.futures import ThreadPoolExecutor

#from tqdm.notebook import tqdm
#from radiant_mlhub import client, get_session

from radiant_mlhub import Dataset, client
import tarfile
from pathlib import Path

###**Authentication**

You need an API_KEY to be able to download some datasets. 

You can get API_KEY from your account on Radiant MLHUB.

If you do not have an account, go to [https://www.mlhub.earth/](https://www.mlhub.earth/) and sign up for API access. 

If you have an account, you can login and get an API_KEY through the API_KEY menu at [https://dashboard.mlhub.earth/](https://dashboard.mlhub.earth/). 

You can then paste your API_KEY in this cell at 'PASTE YOUR API_KEY HERE'.

In [3]:
import os

os.environ['MLHUB_API_KEY'] = 'PASTE YOUR API_KEY HERE'

### **Retrieving the data**

Datasets are stored as collections on Radiant MLHub catalog. A collection represents the top-most data level. Typically this means the data comes from the same source for the same geography. It might include different years or sub-geographies.

The two collections we shall use in the machine learning application are:

- ref_african_crops_kenya_02_source: includes the multi-temporal bands of Sentinel-2
- ref_african_crops_kenya_02_labels: includes the labels and field IDs



In [ ]:
dataset = Dataset.fetch('ref_african_crops_kenya_02')

print(f'ID: {dataset.id}')
print(f'Title: {dataset.title}')
print('Collections:')
for collection in dataset.collections:
    print(f'* {collection.id}')

In [5]:
# output path where you want to download the data
output_path = Path("./data/").resolve()

In [ ]:
archive_paths = dataset.download(output_dir=output_path)
for archive_path in archive_paths:
    print(f'Extracting {archive_path}...')
    with tarfile.open(archive_path) as tfile:
        tfile.extractall(path=output_path)

print('Done\n')

##**Viewing the Images**

In this section, we will explore the downloaded data set by viewing a single image corresponding one spectral band, then we will view an RGB image from a combination of images corresponding to the RGB colors. 

Again, this section of the notebook contains code from the starter notebooks on loading and visualizing a sample of .tif images from Radiant ML Hub. 

In the code, we use the tifffile package to read .tif images. We use the matplotlib library to view the images.

In [7]:
# Required libraries
from pathlib import Path
from io import BytesIO
from glob import glob

from tqdm.notebook import tqdm
import tifffile as tiff
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure

%matplotlib inline

In [8]:
# The path to the source collection that was extracted from the download archive
collection_path = Path('./data/ref_african_crops_kenya_02_source')

In [9]:
def load_file(img_path):
    """Takes a path to the download archive and the path within the archive to the image and returns a numpy array."""
    
    return tiff.imread(str(img_path))

In [10]:
# Get a list of dates that an observation from Sentinel-2 is provided for from the currently 
#  downloaded imagery

tile_dates = dict()

def filter_tifs(name):
    return name.endswith('.tif')

for f in glob(str(collection_path / '**/*.tif')):
    tif_path = Path(f)
    _, tile_id, tile_date = tif_path.parent.name.rsplit('_', 2)

    if tile_id not in tile_dates:
        tile_dates[tile_id] = set()

    tile_dates[tile_id].add(tile_date)

In [ ]:
for tile in sorted(tile_dates.keys()):
    print(f'Tile ID: {tile}')
    dates = sorted(list(tile_dates[tile]))
    print(f'Dates: {", ".join(dates)}')
    print('')

In [12]:
# Available bands
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']

In [13]:
# Sample file to load:
tile = '02'
date_ = '20190825'
band = 'B03'

archive_path = "./data/ref_african_crops_kenya_02_source.tar.gz"
img_path = collection_path / f"ref_african_crops_kenya_02_tile_{tile}_{date_}/{band}.tif"
band_data = load_file(img_path)

In [ ]:
fig = plt.figure(figsize=(7, 7))
plt.imshow(band_data, vmin=0, vmax=0.15)

##**Data preprocessing for machine learning**

Now that you have downloaded and explored the dataset by viewing a sample of images, let's now transform the downloaded data for training and evaluating classificatnon models. 

First of all, we need to get all the dates constituting the observations in the dataset and the bands in each observation into lists that we shall use later to specify images or go through all images.


In [17]:
import datetime
# List of dates that an observation from Sentinel-2 is provided in the training dataset

dates = [datetime.datetime(2019, 6, 6, 8, 10, 7),
         datetime.datetime(2019, 7, 1, 8, 10, 4),
         datetime.datetime(2019, 7, 6, 8, 10, 8),
         datetime.datetime(2019, 7, 11, 8, 10, 4),
         datetime.datetime(2019, 7, 21, 8, 10, 4),
         datetime.datetime(2019, 8, 5, 8, 10, 7),
         datetime.datetime(2019, 8, 15, 8, 10, 6),
         datetime.datetime(2019, 8, 25, 8, 10, 4),
         datetime.datetime(2019, 9, 9, 8, 9, 58),
         datetime.datetime(2019, 9, 19, 8, 9, 59),
         datetime.datetime(2019, 9, 24, 8, 9, 59),
         datetime.datetime(2019, 10, 4, 8, 10),
         datetime.datetime(2019, 11, 3, 8, 10)]

In [18]:
# List of bands in each observation

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']

###**Getting data for each pixel**

This is where we start massaging the data into a format we can use.

We start by reading in the labels, finding the pixel locations of all the fields (most are only a few pixels in size) and storing the pixel locations, the field ID and the label.

In [ ]:
# Not super efficient but  ¯\_(ツ)_/¯
import pandas as pd

row_locs = []
col_locs = []
field_ids = []
labels = []
tiles = []
for tile in range(4):
  fids = f'./data/ref_african_crops_kenya_02_labels/ref_african_crops_kenya_02_tile_0{tile}_label/field_ids.tif'
  labs = f'./data/ref_african_crops_kenya_02_labels/ref_african_crops_kenya_02_tile_0{tile}_label/labels.tif'
  fid_arr = load_file(fids)
  lab_arr = load_file(labs)
  for row in range(len(fid_arr)):
    for col in range(len(fid_arr[0])):
      if fid_arr[row][col] != 0:
        row_locs.append(row)
        col_locs.append(col)
        field_ids.append(fid_arr[row][col])
        labels.append(lab_arr[row][col])
        tiles.append(tile)

df = pd.DataFrame({
    'fid':field_ids,
    'label':labels,
    'row_loc': row_locs,
    'col_loc':col_locs,
    'tile':tiles
})

print(df.shape)
print(df.groupby('fid').count().shape)
df.head()

---

Then we loop through all the images, sampling the different image bands to get the values for each pixel in each field.

---

In [ ]:
# Sample the bands at different dates as columns in our new dataframe
col_names = []
col_values = []

for tile in range(4): # 1) For each tile
  print('Tile: ', tile)
  for d in dates: # 2) For each date
    print(str(d))
    d = ''.join(str(d.date()).split('-')) # Nice date string
    t = '0'+str(tile)
    for b in bands: # 3) For each band
      col_name = d+'_'+b
      
      if tile == 0:
        # If the column doesn't exist, create it and populate with 0s
        df[col_name] = 0

      # Load im
      im = load_file(f"./data/ref_african_crops_kenya_02_source/ref_african_crops_kenya_02_tile_{t}_{d}/{b}.tif")
      
      # Going four levels deep. Each second on the outside is four weeks in this loop
      # If we die here, there's no waking up.....
      vals = []
      for row, col in df.loc[df.tile == tile][['row_loc', 'col_loc']].values: # 4) For each location of a pixel in a field
        vals.append(im[row][col])
      df.loc[df.tile == tile, col_name] = vals
df.head()


In [ ]:
df.to_csv('sampled_data.csv', index=False)

---

We can copy the prepared data (sampled_data.csv) to the data folder in Google Drive

---

In [ ]:
!cp 'sampled_data.csv' '/content/drive/My Drive/Colab Notebooks/data/sampled_data.csv' # You can mount drive and save like this, or just download sampled_data.csv and use that later.

# **Using the prepared data for machine learning crop type classification models**

---

Now that we have our data in a nice tabular data format, we can try some simple machine learning methods using the sklearn package.

We can restart the runtime here and load in the data we saved earlier.

---

### **Import classifiers from sklearn**


In [36]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier # Random Forest classifier, AdaBoost classifier
from sklearn.neighbors import KNeighborsClassifier # K nearest neighbor classifier
from sklearn.tree import DecisionTreeClassifier # Decision tree classifier
from sklearn.gaussian_process import GaussianProcessClassifier # Gaussian process classifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import log_loss # evaluation metric

### **Load data set**

We load the data set and split it into a training set (for training a classifier) and a test set (for evaluating the classifier).


In [ ]:
!ls

In [ ]:
# Load the data
df = pd.read_csv('./sampled_data.csv')
print(df.shape)
df.head()

In [39]:
# Split into train and test sets
train = df.loc[df.label != 0]
test =  df.loc[df.label == 0]
train.shape, test.shape

((47791, 174), (19766, 174))

In [40]:
# Split train into train and validation (val) set, so that we can score our models locally.

# We split on field ID since we want to predict for unseen FIELDS
val_field_ids = train.groupby('fid').mean().reset_index()['fid'].sample(frac=0.3).values
tr = train.loc[~train.fid.isin(val_field_ids)].copy()
val = train.loc[train.fid.isin(val_field_ids)].copy()


### **Classification model training**

We can train a classifier to predict on pixels and then combine, or we can train a classfier by first grouping into fields. Let's do both and compare!

**A. Training and predicting on pixels**

In [41]:
# Split into X and Y for modelling
X_train, y_train = tr[tr.columns[5:]], tr['label']
X_val, y_val = val[val.columns[5:]], val['label']

In [ ]:
# Train a classification model (takes a few minutes since we have plenty of rows)
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train.fillna(0), y_train)

In [ ]:
# Predict on pixels

# We use the predict_proba() function to get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val dataframe as columns
for i in range(7):
  val[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
  val['true'+str(c)] = (val['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]

# Calculate the evaluation score - using the log-loss metric
#print('Pixel score:', log_loss(val[true_cols], val[pred_cols]))
print('Field score:', log_loss(val.groupby('fid').mean()[true_cols], val.groupby('fid').mean()[pred_cols])) # This is what we'll use to compare

# Inspect or compare visually
val[['label']+true_cols+pred_cols].head()

```
# This is formatted as code
```

**B. Training and predicting on fields**

In [44]:
# Group training data into fields

train_grouped = tr.groupby('fid').mean().reset_index()
val_grouped = val.groupby('fid').mean().reset_index()
X_train, y_train = train_grouped[train_grouped.columns[5:]], train_grouped['label']
X_val, y_val = val_grouped[train_grouped.columns[5:]], val_grouped['label']

In [ ]:
# Train a classification model
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train.fillna(0), y_train)

In [ ]:
# Get predicted probabilities
preds = model.predict_proba(X_val.fillna(0))

# Add to val_grouped dataframe as columns
for i in range(7):
  val_grouped[str(i+1)] = preds[:,i]

# Get 'true' vals as columns in val
for c in range(1, 8):
  val_grouped['true'+str(c)] = (val_grouped['label'] == c).astype(int)

pred_cols = [str(i) for i in range(1, 8)]
true_cols = ['true'+str(i) for i in range(1, 8)]
val_grouped[['label']+true_cols+pred_cols].head()

# Already grouped, but just to double check:
print('Field score:', log_loss(val_grouped.groupby('fid').mean()[true_cols], val_grouped.groupby('fid').mean()[pred_cols]))

## **PRACTICAL EXERCISE**

### **If you were able to complete train and predict in A and B, answer the following questions**

#### **Qn 1.** Which of the two approaches is much faster?

#### **Qn 2.** Which of the two approaches is more accurate?

#### **Qn 3 - Hands on.** Instead of the Random Forest classifier, apply other classifiers from the sklearn package and compare their performance using the log loss metric.